In [18]:
# 필요 모듈

import numpy as np

In [19]:
# gamecomponent.py

class GameComponent:
    def __init__(self, verbose=False):
        self.verbose = verbose

In [20]:
# yut.py

class Yut(GameComponent):
    def __init__(self, verbose=False):
        super().__init__(verbose)
        self.yut_myturn = []
        self.throw_result = None

        self.y_do = '''
┌---┐┌---┐┌---┐┌---┐
| x || x || x ||   |
|   ||   ||   ||   |
| x || x || x ||   |
|   ||   ||   ||   |
| x || x || x ||   |
└---┘└---┘└---┘└---┘
'''
        self.y_gae = '''
┌---┐┌---┐┌---┐┌---┐
| x || x ||   ||   |
|   ||   ||   ||   |
| x || x ||   ||   |
|   ||   ||   ||   |
| x || x ||   ||   |
└---┘└---┘└---┘└---┘
'''
        self.y_gul = '''
┌---┐┌---┐┌---┐┌---┐
| x ||   ||   ||   |
|   ||   ||   ||   |
| x ||   ||   ||   |
|   ||   ||   ||   |
| x ||   ||   ||   |
└---┘└---┘└---┘└---┘
'''
        self.y_yut = '''
┌---┐┌---┐┌---┐┌---┐
|   ||   ||   ||   |
|   ||   ||   ||   |
|   ||   ||   ||   |
|   ||   ||   ||   |
|   ||   ||   ||   |
└---┘└---┘└---┘└---┘
'''
        self.y_mo = '''
┌---┐┌---┐┌---┐┌---┐
| x || x || x || x |
|   ||   ||   ||   |
| x || x || x || x |
|   ||   ||   ||   |
| x || x || x || x |
└---┘└---┘└---┘└---┘
'''
        self.y_backdo = '''
┌---┐┌---┐┌---┐┌---┐
|   || x || x || x |
|   ||   ||   ||   |
| * || x || x || x |
|   ||   ||   ||   |
|   || x || x || x |
└---┘└---┘└---┘└---┘
'''

    def throw(self, allow_backdo=True):
        yut_choice = ['도', '개', '걸', '윷', '모', '빽도']
        yut_prob = [0.1875, 0.375, 0.25, 0.0625, 0.0625, 0.0625]

        if not allow_backdo:
            yut_choice = ['도', '개', '걸', '윷', '모']
            yut_prob = [0.1875, 0.375, 0.25, 0.0625, 0.0625]
            total_prob = sum(yut_prob)
            yut_prob = [p / total_prob for p in yut_prob]

        self.throw_result = np.random.choice(yut_choice, p=yut_prob)
        self.yut_myturn.append(self.throw_result)
        print(f'윷을 던져 {self.throw_result}가 나왔습니다.')

        if self.throw_result == '도':
            print(self.y_do)
        elif self.throw_result == '개':
            print(self.y_gae)
        elif self.throw_result == '걸':
            print(self.y_gul)
        elif self.throw_result == '윷':
            print(self.y_yut)
        elif self.throw_result == '모':
            print(self.y_mo)
        elif self.throw_result == '빽도':
            print(self.y_backdo)

        return self.throw_result

    def yut_list(self):
        return self.yut_myturn

    def reset_yut(self):
        self.yut_myturn = []
        self.throw_result = None

In [21]:
# setmal.py

class SetMal(GameComponent):
    def __init__(self, player, mal, verbose=False):
        super().__init__(verbose)
        self.player = player
        self.mal = mal
        self.position = -1  # -1 represents off the board
        self.is_finished = False  # 골인 여부
        self.stacked = False
        self.stack_mal = mal
        self.current_route = None
        self.is_stack_leader = True
        self.route_name = 'default'

    def print_symbol(self):
        if self.stacked and self.is_stack_leader:
            return self.stack_mal.upper()  # 스택 리더는 대문자로 표시
        elif not self.stacked and not self.is_finished:
            return self.mal
        else:
            return None

In [22]:
# mal.py

class Mal(SetMal):
    def __init__(self, player, mal, verbose=False):
        super().__init__(player, mal, verbose)

        # 백도 이동 정의 (모든 위치 포함)
        self.backdo_moves = {
            0: 1,
            1: 2,
            2: 3,
            3: 4,
            4: 5,
            5: 9,
            6: 0,
            7: 0,
            8: 5,
            9: 13,
            10: 6,
            11: 7,
            12: 8,
            13: 18,
            14: 11,
            15: 10,
            16: 14,
            17: 14,
            18: 22,
            19: 15,
            20: 16,
            21: 17,
            22: 28,
            23: 19,
            24: 23,
            25: 24,
            26: 25,
            27: 26,
            28: 21
            # 29 (GOAL)은 따로 처리
        }

        # 초기 위치 매핑 정의
        self.initial_moves = {
            '도': 22,
            '개': 18,
            '걸': 13,
            '윷': 9,
            '모': 5
        }

    def mal_move_value(self, yut_result):
        if yut_result == '도':
            return 1
        elif yut_result == '개':
            return 2
        elif yut_result == '걸':
            return 3
        elif yut_result == '윷':
            return 4
        elif yut_result == '모':
            return 5
        elif yut_result == '빽도':
            return -1
        return 0

    def select_route(self, yutpan):
        default_route = yutpan.routes['default']
        from_5 = yutpan.routes['from_5']
        from_0 = yutpan.routes['from_0']
        from_14 = yutpan.routes['from_14']

        pos = self.position

        def in_route(p, r):
            return p in r

        if pos == -1:
            # 보드 외부에서 시작, 기본 경로 설정
            self.current_route = default_route
            self.route_name = 'default'
            return

        # from_5 경로 끝(23)에 도달 시 default 경로로 합류
        if self.route_name == 'from_5' and pos == 23:
            if 23 in default_route:
                self.route_name = 'default'
                self.current_route = default_route
            return

        # from_0과 from_14 경로 끝(29)에 도달 시 default 경로로 합류
        if self.route_name == 'from_0' and pos >= 29:
            if 29 in default_route:
                self.route_name = 'default'
                self.current_route = default_route
            return

        if self.route_name == 'from_14' and pos >= 29:
            if 29 in default_route:
                self.route_name = 'default'
                self.current_route = default_route
            return

        # 특정 위치에서 새로운 경로 설정
        if pos == 5:
            self.route_name = 'from_5'
            self.current_route = from_5
            return
        elif pos == 0:
            self.route_name = 'from_0'
            self.current_route = from_0
            return
        elif pos == 14:
            self.route_name = 'from_14'
            self.current_route = from_14
            return

        # 현재 route_name에 따라 route 결정
        if self.route_name == 'default':
            route = default_route
        elif self.route_name == 'from_5':
            route = from_5
        elif self.route_name == 'from_0':
            route = from_0
        elif self.route_name == 'from_14':
            route = from_14
        else:
            route = default_route

        if not in_route(pos, route):
            if in_route(pos, from_5):
                self.route_name = 'from_5'
                route = from_5
            elif in_route(pos, from_0):
                self.route_name = 'from_0'
                route = from_0
            elif in_route(pos, from_14):
                self.route_name = 'from_14'
                route = from_14
            elif in_route(pos, default_route):
                self.route_name = 'default'
                route = default_route
            else:
                self.route_name = 'default'
                route = default_route

        self.current_route = route

    def move(self, steps, yutpan, game=None, yut_result=None):
        if self.is_finished:
            if self.verbose:
                print(f"[DEBUG] {self.player}님의 말 '{self.mal}'은 이미 골인했습니다.")
            return False  # 골인된 말은 이동 불가

        if yut_result == '빽도':
            if self.position in self.backdo_moves:
                new_pos = self.backdo_moves[self.position]
                self.position = new_pos
                self.select_route(yutpan)
                if self.verbose:
                    print(f"[DEBUG] {self.player}님의 말 '{self.mal}'이 빽도에 의해 인덱스 {new_pos}로 이동했습니다.")
                return False
            else:
                if self.verbose:
                    print(f"[DEBUG] {self.player}님의 말 '{self.mal}'은 현재 위치에서 빽도를 사용할 수 없습니다.")
                return False  # 빽도 사용 불가

        if self.position == -1 and yut_result in self.initial_moves:
            self.position = self.initial_moves[yut_result]
            self.select_route(yutpan)
            if self.verbose:
                print(f"[DEBUG] {self.player}님의 말 '{self.mal}'이 출발했습니다. 인덱스 {self.position}")
            return False  # 초기 이동 후 추가 이동 없음

        if self.position == -1 and steps > 0:
            # 말이 출발하지 않은 상태에서 움직일 때
            self.position = yutpan.routes['default'][0]  # 22로 이동
            self.select_route(yutpan)
            if self.verbose:
                print(f"[DEBUG] {self.player}님의 말 '{self.mal}'이 출발했습니다. 인덱스 {self.position}")
            steps -= 1  # 남은 이동 칸수 감소

        while steps > 0 and not self.is_finished:
            route = self.current_route
            try:
                current_idx = route.index(self.position)
            except ValueError:
                if self.verbose:
                    print(f"Error: Position {self.position} not found in the current route.")
                return False

            new_idx = current_idx + steps
            last_idx = len(route) - 1

            if self.verbose:
                print(f"[DEBUG] {self.player}님의 말 '{self.mal}'을 {steps}칸 이동합니다. 현재 인덱스: {current_idx}, 새 인덱스: {new_idx}")

            if new_idx > last_idx:
                if self.current_route == yutpan.routes['default']:
                    # 골인 처리
                    self.position = -1
                    self.is_finished = True
                    print(f"{self.player}님의 말 '{self.mal}'이 골인했습니다!")

                    # 스택된 말도 골인 처리
                    if self.stacked and game is not None:
                        for m in game.players_mal_dict[self.player]:
                            if m.stack_mal.upper() == self.stack_mal.upper():
                                m.is_finished = True
                                if self.verbose:
                                    print(f"[DEBUG] {self.player}님의 말 '{m.mal}'도 골인 처리되었습니다.")
                    return True
                else:
                    # 경로 전환
                    steps_to_end = last_idx - current_idx
                    self.position = route[last_idx]
                    steps_remaining = steps - (steps_to_end + 1)
                    self.select_route(yutpan)
                    if self.verbose:
                        print(f"[DEBUG] {self.player}님의 말 '{self.mal}'이 경로 끝에 도달하여 default 경로로 전환됩니다.")
                    steps = steps_remaining
            elif new_idx == last_idx:
                if self.current_route == yutpan.routes['default']:
                    # 골인 처리
                    self.position = -1
                    self.is_finished = True
                    print(f"{self.player}님의 말 '{self.mal}'이 골인했습니다!")

                    # 스택된 말도 골인 처리
                    if self.stacked and game is not None:
                        for m in game.players_mal_dict[self.player]:
                            if m.stack_mal.upper() == self.stack_mal.upper():
                                m.is_finished = True
                                if self.verbose:
                                    print(f"[DEBUG] {self.player}님의 말 '{m.mal}'도 골인 처리되었습니다.")
                    return True
                else:
                    # 경로 전환
                    self.position = route[new_idx]
                    self.select_route(yutpan)
                    if self.verbose:
                        print(f"[DEBUG] {self.player}님의 말 '{self.mal}'이 경로 끝에 도달하여 default 경로로 전환됩니다.")
                    steps = 0  # 남은 이동 칸수 없음
            else:
                self.position = route[new_idx]
                self.select_route(yutpan)  # 이동 후 경로 재선택
                if self.verbose:
                    print(f"[DEBUG] {self.player}님의 말 '{self.mal}'이 인덱스 {new_idx}로 이동했습니다.")
                steps = 0  # 남은 이동 칸수 없음

        # 추가적인 골인 처리 확인 (position >=29)
        if self.position >= 29:
            self.position = -1
            self.is_finished = True
            print(f"{self.player}님의 말 '{self.mal}'이 골인했습니다!")

            # 스택된 말도 골인 처리
            if self.stacked and game is not None:
                for m in game.players_mal_dict[self.player]:
                    if m.stack_mal.upper() == self.stack_mal.upper():
                        m.is_finished = True
                        if self.verbose:
                            print(f"[DEBUG] {self.player}님의 말 '{m.mal}'도 골인 처리되었습니다.")
            return True

        return False

    def mal_catch(self, other_mal_list, game):
        if self.is_finished:
            return False  # 골인된 말은 잡을 수 없음

        caught = False
        for om in other_mal_list:
            if om.is_finished:
                continue  # 골인된 말은 보드에 없음
            if om.position == self.position and om.player != self.player:
                if self.verbose:
                    print(f"[DEBUG] {self.player}님의 말 '{self.mal}'이 {om.player}님의 말 '{om.mal}'을 잡았습니다!")
                print('말을 잡았습니다!')
                caught = True
                if om.stacked:
                    self.unstack_captured(om, game)
                else:
                    om.position = -1
        return caught

    def mal_stack(self, own_mal_list):
        if self.is_finished:
            if self.verbose:
                print(f"[DEBUG] {self.player}님의 말 '{self.mal}'은 골인되어 스택할 수 없습니다.")
            return False  # 골인된 말은 스택할 수 없음

        for om in own_mal_list:
            if om != self and om.player == self.player and om.position == self.position and om.position != -1 and not om.is_finished:
                if self.verbose:
                    print(f"[DEBUG] {self.player}님의 말 '{self.mal}'과 '{om.mal}'이 스택됩니다.")
                self.stacked = True
                om.stacked = True
                self.stack_mal = self.mal.upper()
                om.stack_mal = self.stack_mal  # 팔로워의 stack_mal을 리더와 동일하게 설정
                om.position = -1
                om.is_stack_leader = False
                return True
        return False

    def unstack_captured(self, om, game):
        player = om.player
        mals = game.players_mal_dict[player]
        for m in mals:
            if m.stack_mal.upper() == om.stack_mal.upper():
                m.position = -1
                m.stacked = False
                m.is_stack_leader = True
                m.is_finished = False  # 필요 시 골인 상태 초기화
                if self.verbose:
                    print(f"[DEBUG] {player}님의 말 '{m.mal}'이 캡처되어 시작 전 상태로 돌아갔습니다.")

In [23]:
# player.py

class Player(GameComponent):
    def __init__(self, name, mal_symbols, verbose=False):
        super().__init__(verbose)
        self.name = name
        self.mal_symbols = mal_symbols
        self.mals = [Mal(name, symbol, verbose) for symbol in mal_symbols]

    def get_active_mals(self):
        return [mal for mal in self.mals if not mal.is_finished]

In [24]:
# yutpan.py

class Yutpan(GameComponent):
    def __init__(self, verbose=False):
        super().__init__(verbose)
        self.board_matrix = [
            ["□", "□", "□", " ", "□", "□", "□", " "],
            ["□", "□", " ", " ", " ", "□", "□", " "],
            ["□", " ", "□", " ", "□", " ", "□", " "],
            [" ", " ", " ", "□", " ", " ", " ", " "],
            ["□", " ", "□", " ", "□", " ", "□", " "],
            ["□", "□", " ", " ", " ", "□", "□", " "],
            ["□", "□", "□", " ", "□", "□", "□", "GOAL"]
        ]
        self.rows = 7
        self.cols = 8  # GOAL을 추가하여 열 수를 8로 설정

        self.squares = []
        for r in range(self.rows):
            for c in range(self.cols):
                if self.board_matrix[r][c] == "□":
                    self.squares.append((r, c))
                elif self.board_matrix[r][c] == "GOAL":
                    self.squares.append("GOAL")  # GOAL은 문자열로 추가

        if self.verbose:
            print(f"[DEBUG] 총 squares 개수: {len(self.squares)}")

        # 시작 위치를 보드 외부의 특별한 인덱스로 설정 (-1)
        # routes['default']는 squares 리스트의 인덱스를 사용
        self.routes = {
            'default': [22, 18, 13, 9, 5, 4, 3, 2, 1, 0, 6, 10, 15, 19, 23, 24, 25, 26, 27, 28, 29],  # 29은 GOAL
            'from_5': [5, 8, 12, 14, 16, 20, 23],
            'from_0': [0, 7, 11, 14, 17, 21, 28, 29],       # 29 추가
            'from_14': [14, 17, 21, 28, 29]                 # 29 추가
        }
        if self.verbose:
            print(f"[DEBUG] Route 정의: {self.routes}")

    def print_board(self, player_mals):
        temp_board = [row[:] for row in self.board_matrix]

        for p, mals in player_mals.items():
            for m in mals:
                if m.position != -1 and not m.is_finished:
                    # GOAL 위치에는 말을 표시하지 않음
                    if m.position == 29:
                        continue
                    if isinstance(m.position, tuple):
                        r, c = m.position
                    else:
                        if m.position >= len(self.squares) or self.squares[m.position] == "GOAL":
                            continue  # 유효하지 않은 위치는 무시
                        pos = self.squares[m.position]
                        if isinstance(pos, tuple):
                            r, c = pos
                        else:
                            continue  # GOAL은 표시하지 않음
                    symbol = m.print_symbol()
                    if symbol is not None:
                        temp_board[r][c] = symbol.upper() if m.stacked and m.is_stack_leader else symbol  # 스택 리더는 대문자로 표시

        print('\n현재 윷판 상태:')
        for row in temp_board:
            print(' '.join(row))
        print()

    def is_all_goal(self, mals):
        return all(m.is_finished for m in mals)  # 모든 말이 골인했는지 확인

In [25]:
# gamestart.py

class Gamestart(GameComponent):
    def __init__(self, verbose=False):
        super().__init__(verbose)
        self.player1 = self.get_unique_player_name(1)
        self.player2 = self.get_unique_player_name(2)
        self.players_info = {}
        self.players_mal_dict = {}
        self.current_player = self.player1

    def get_unique_player_name(self, player_number):
        while True:
            name = input(f'{player_number}번 플레이어의 이름을 입력하세요 : ').strip()
            if not name:
                print("이름을 입력해야 합니다. 다시 입력하세요.")
                continue
            # 플레이어2일 때 플레이어1과 이름 중복 체크
            if player_number == 2 and name == self.player1:
                print("플레이어1과 동일한 이름입니다. 다른 이름을 입력하세요.")
                continue
            return name

    def set_mal(self):
        while True:
            try:
                mal1 = input(f'{self.player1}님이 사용할 말을 소문자 영어 한 글자로 입력하세요 : ').strip()
                if len(mal1) != 1 or not mal1.isalpha() or not mal1.islower():
                    raise ValueError('말은 소문자 영어 한 글자로 입력해야합니다.')

                mal2 = input(f'{self.player2}님이 사용할 말을 소문자 영어 한 글자로 입력하세요 : ').strip()
                if len(mal2) != 1 or not mal2.isalpha() or not mal2.islower():
                    raise ValueError('말은 소문자 영어 한 글자로 입력해야합니다.')

                # 말 심볼 중복 체크
                if mal2 == mal1:
                    raise ValueError('플레이어1과 플레이어2의 말 심볼이 동일합니다. 다른 심볼을 선택하세요.')

                self.players_info[self.player1] = [mal1, mal1]
                self.players_info[self.player2] = [mal2, mal2]
                break
            except ValueError as e:
                print(e)

        # 플레이어 객체 생성
        player1_obj = Player(self.player1, self.players_info[self.player1], self.verbose)
        player2_obj = Player(self.player2, self.players_info[self.player2], self.verbose)

        self.players_mal_dict[self.player1] = player1_obj.mals
        self.players_mal_dict[self.player2] = player2_obj.mals

        return self.players_info

    def switch_player(self):
        self.current_player = self.player1 if self.current_player == self.player2 else self.player2

In [26]:
# myturn.py

class Myturn(GameComponent):
    def __init__(self, game, yutpan, verbose=False):
        super().__init__(verbose)
        self.game = game
        self.yutpan = yutpan
        self.yut = Yut(verbose)

    def prompt_throw(self):
        tries = 0
        while True:
            user_input = input('Enter 키로 윷을 던지세요! : ')
            try:
                if user_input != '':
                    raise ValueError('Enter 키가 아닌 다른 키를 입력했습니다!')
                return True
            except ValueError as e:
                print(e)
                tries += 1
                if tries == 1:
                    print('한 번 더 잘못 던지면 낙입니다!')
                else:
                    print('낙입니다. 다음 차례로 넘어갑니다.')
                    return False

    def playmyturn(self):
        self.yut.reset_yut()

        # 백도 선택 가능 여부 결정
        current_player = self.game.current_player
        current_mals = self.game.players_mal_dict[current_player]
        allow_backdo = any(m.position != -1 and not m.is_finished for m in current_mals)

        if not self.prompt_throw():
            return None
        result = self.yut.throw(allow_backdo=allow_backdo)
        if result is None:
            return None

        while result in ['윷', '모']:
            print("윷/모 나왔으니 한 번 더 던지세요!")
            if not self.prompt_throw():
                return self.yut.yut_list()
            result = self.yut.throw(allow_backdo=allow_backdo)
            if result is None:
                return None

        return self.yut.yut_list()

    def extra_throw(self):
        self.yut.reset_yut()
        # 백도 선택 가능 여부 결정
        current_player = self.game.current_player
        current_mals = self.game.players_mal_dict[current_player]
        allow_backdo = any(m.position != -1 and not m.is_finished for m in current_mals)

        if not self.prompt_throw():
            return None
        result = self.yut.throw(allow_backdo=allow_backdo)
        if result is None:
            return None
        while result in ['윷', '모']:
            print("윷/모 나왔으니 한 번 더 던지세요!")
            if not self.prompt_throw():
                return self.yut.yut_list()
            result = self.yut.throw(allow_backdo=allow_backdo)
            if result is None:
                return None
        return self.yut.yut_list()

    def choose_result(self, results, allow_backdo):
        available_results = results.copy()
        if not allow_backdo and '빽도' in available_results:
            available_results.remove('빽도')

        while True:
            print("현재 사용 가능한 윷 결과:", available_results)
            choice = input("사용할 윷 결과를 입력하세요(도/개/걸/윷/모/빽도): ").strip()
            if choice in available_results:
                available_results.remove(choice)
                results.remove(choice)  # 원본 리스트에서도 제거
                return choice
            else:
                print("해당 윷 결과는 사용 가능한 윷 결과 목록에 없습니다. 다시 입력하세요.")

    def turn_process(self):
        initial_results = self.playmyturn()
        if initial_results is None or len(initial_results) == 0:
            return

        results = initial_results[:]
        current_player = self.game.current_player
        current_mals = self.game.players_mal_dict[current_player]
        other_player = self.game.player1 if current_player == self.game.player2 else self.game.player2
        other_mals = self.game.players_mal_dict[other_player]

        # 백도 선택 가능 여부 결정
        allow_backdo = any(m.position != -1 and not m.is_finished for m in current_mals)

        while len(results) > 0:
            selected_result = self.choose_result(results, allow_backdo)

            # 골인된 말 제외
            stack_leader = [m for m in current_mals if m.position != -1 and m.stacked and m.is_stack_leader and not m.is_finished]
            alive_mals = [m for m in current_mals if (m.position != -1 and not m.is_finished) or (m.position == -1 and not m.is_finished and not m.stacked and m.is_stack_leader)]

            if stack_leader:
                selected_mal = stack_leader[0]
            else:
                if len(alive_mals) == 1:
                    selected_mal = alive_mals[0]
                else:
                    if selected_result == '빽도':
                        selectable = [m for m in current_mals if m.position != -1 and not m.is_finished]
                        if len(selectable) == 0:
                            print('이동 가능한 말이 없습니다. 다음 차례로 넘어갑니다.')
                            return
                        print("현재 플레이어 말 상태(빽도 - 보드 위 말만 선택 가능):")
                        for idx, mm in enumerate(current_mals):
                            if mm in selectable:
                                pos_str = f"인덱스 {mm.position}"
                                print(f"{idx+1}. {mm.mal} ({pos_str})")
                        while True:
                            try:
                                choice = input("이동시킬 말 번호를 입력하세요(빽도): ")
                                if not choice.isdigit():
                                    raise ValueError("숫자를 입력하세요.")
                                c_idx = int(choice) - 1
                                if c_idx < 0 or c_idx >= len(current_mals):
                                    raise ValueError("잘못된 말 번호.")
                                selected_mal = current_mals[c_idx]
                                if selected_mal not in selectable:
                                    raise ValueError("빽도는 보드 위에 있는 말만 선택할 수 있습니다.")
                                break
                            except ValueError as e:
                                print(e)
                    else:
                        selectable = [m for m in current_mals if m.position != -1 and not m.is_finished]
                        not_started = [m for m in current_mals if m.position == -1 and not m.is_finished]

                        if len(selectable) == 0 and len(not_started) == 1:
                            selected_mal = not_started[0]
                        else:
                            print("현재 플레이어 말 상태:")
                            for idx, mm in enumerate(current_mals):
                                if mm.is_finished:
                                    continue  # 골인된 말 제외
                                pos_str = "시작 전" if mm.position == -1 else f"인덱스 {mm.position}"
                                print(f"{idx+1}. {mm.mal} ({pos_str})")
                            while True:
                                try:
                                    choice = input("이동시킬 말 번호를 입력하세요: ")
                                    if not choice.isdigit():
                                        raise ValueError("숫자를 입력하세요.")
                                    c_idx = int(choice) - 1
                                    if c_idx < 0 or c_idx >= len(current_mals):
                                        raise ValueError("잘못된 말 번호.")
                                    selected_mal = current_mals[c_idx]
                                    if selected_mal.is_finished:
                                        raise ValueError("이미 골인한 말은 선택할 수 없습니다.")
                                    break
                                except ValueError as e:
                                    print(e)

            if selected_mal.is_finished:
                print(f"선택한 말 '{selected_mal.mal}'은 이미 골인했습니다. 다른 말을 선택하세요.")
                continue  # 다음 반복으로 이동

            steps = selected_mal.mal_move_value(selected_result)

            if selected_result == '빽도' and all(m.position == -1 or m.is_finished for m in current_mals):
                print('이동 가능한 말이 없습니다. 다음 차례로 넘어갑니다.')
                return

            is_goal = selected_mal.move(steps, self.yutpan, game=self.game, yut_result=selected_result)

            caught = selected_mal.mal_catch(other_mal_list=other_mals, game=self.game)
            if caught:
                extra_res = self.extra_throw()
                if extra_res is None:
                    return
                else:
                    results.extend(extra_res)

            # 스택 처리
            selected_mal.mal_stack(own_mal_list=current_mals)
            self.yutpan.print_board(self.game.players_mal_dict)

            if self.yutpan.is_all_goal(current_mals):
                print(f"{current_player}님이 모든 말을 골인시켰습니다! 승리!")
                return "WIN"

        return "CONTINUE"

In [27]:
# 메인 함수 - 게임 실행
def main():
    verbose = False  # 디버그 출력을 원하지 않으면 False로 설정
    game = Gamestart(verbose)
    game.set_mal()
    yutpan = Yutpan(verbose)

    print('윷놀이를 시작합니다!')
    print('(참고 : 윷판에 해당하는 인덱스는 좌측 상단부터 우측 하단 순서로 실행됩니다!)')

    while True:
        print(f"{game.current_player}님의 차례입니다.")
        yutpan.print_board(game.players_mal_dict)

        turn = Myturn(game, yutpan, verbose)
        result = turn.turn_process()

        if result == "WIN":
            print(f"{game.current_player}님이 게임에서 승리하셨습니다! 축하드립니다!")
            break
        elif result in [None, "CONTINUE"]:
            game.switch_player()

if __name__ == "__main__":
    main()


1번 플레이어의 이름을 입력하세요 : 성민
2번 플레이어의 이름을 입력하세요 : 성민
플레이어1과 동일한 이름입니다. 다른 이름을 입력하세요.
2번 플레이어의 이름을 입력하세요 : 민지
성민님이 사용할 말을 소문자 영어 한 글자로 입력하세요 : a
민지님이 사용할 말을 소문자 영어 한 글자로 입력하세요 : A
말은 소문자 영어 한 글자로 입력해야합니다.
성민님이 사용할 말을 소문자 영어 한 글자로 입력하세요 : a
민지님이 사용할 말을 소문자 영어 한 글자로 입력하세요 : a
플레이어1과 플레이어2의 말 심볼이 동일합니다. 다른 심볼을 선택하세요.
성민님이 사용할 말을 소문자 영어 한 글자로 입력하세요 : a
민지님이 사용할 말을 소문자 영어 한 글자로 입력하세요 : b
윷놀이를 시작합니다!
(참고 : 윷판에 해당하는 인덱스는 좌측 상단부터 우측 하단 순서로 실행됩니다!)
성민님의 차례입니다.

현재 윷판 상태:
□ □ □   □ □ □  
□ □       □ □  
□   □   □   □  
      □        
□   □   □   □  
□ □       □ □  
□ □ □   □ □ □ GOAL

Enter 키로 윷을 던지세요! : 
윷을 던져 개가 나왔습니다.

┌---┐┌---┐┌---┐┌---┐
| x || x ||   ||   |
|   ||   ||   ||   |
| x || x ||   ||   |
|   ||   ||   ||   |
| x || x ||   ||   |
└---┘└---┘└---┘└---┘

현재 사용 가능한 윷 결과: ['개']
사용할 윷 결과를 입력하세요(도/개/걸/윷/모/빽도): 개
현재 플레이어 말 상태:
1. a (시작 전)
2. a (시작 전)
이동시킬 말 번호를 입력하세요: 1

현재 윷판 상태:
□ □ □   □ □ □  
□ □       □ □  
□   □   □   □  
      □        
□   □   □   a  
□ □       □ □  
□ □ □   □ □ □ GOA